<a href="https://colab.research.google.com/github/RajeshworM/Yield_Modelling_Automation/blob/main/IMDGRD_Weather_DailyDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='green'> <font size ='5'> **Realtime Districts Daily temperature (Minimum) data download for grid data**

In [ ]:
# Colab-ready: download IMD .grd daily files, map grid cells -> all districts, produce district-wise CSV
# Run this in Google Colab. Adjust start_date / end_date and paths as needed.

# 0) Install dependencies (may take a minute)
!pip install -q geopandas requests tqdm shapely pyproj fiona

import os
import zipfile
import requests
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from tqdm import tqdm

import geopandas as gpd
from shapely.geometry import Point

# ------------------ CONFIG ------------------
IMD_BASE = "https://www.imdpune.gov.in/cmpg/Realtimedata/min/"
CTL_URL  = IMD_BASE + "TEMP.CTL"
GRD_FILENAME_PATTERN = "min{dd}{mm}{yyyy}.grd"

GEOBOUNDARIES_IND_ADM2_ZIP = "https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/IND/ADM2/geoBoundaries-IND-ADM2-all.zip"

WORKDIR = "/content/imdpune_min"
GRD_DIR = os.path.join(WORKDIR, "grds")
GEO_DIR = os.path.join(WORKDIR, "geoboundaries")
OUT_CSV = os.path.join(WORKDIR, "district_daily_min_temp_2025.csv")

os.makedirs(WORKDIR, exist_ok=True)
os.makedirs(GRD_DIR, exist_ok=True)
os.makedirs(GEO_DIR, exist_ok=True)

start_date = datetime(2025, 1, 1)
end_date   = datetime.today()

small_test = False
if small_test:
    end_date = start_date + timedelta(days=2)

# ------------------ UTILITY ------------------
def download_file(url, out_path, stream=True, overwrite=False):
    if os.path.exists(out_path) and not overwrite:
        print("Already have", out_path)
        return out_path
    print("Downloading:", url)
    r = requests.get(url, stream=stream, timeout=60)
    if r.status_code != 200:
        print(f"Failed to download {url} (status {r.status_code})")
        return None
    with open(out_path, "wb") as fh:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                fh.write(chunk)
    print("Saved ->", out_path)
    return out_path

# ------------------ 1) Download and parse CTL ------------------
ctl_local = os.path.join(WORKDIR, "TEMP.CTL")
download_file(CTL_URL, ctl_local)

meta = {}
with open(ctl_local, "rt") as f:
    for line in f:
        parts = line.strip().split()
        if not parts: continue
        key = parts[0].upper()
        if key == "XDEF":
            meta["ncols"] = int(parts[1])
            meta["x_start"] = float(parts[3])
            meta["x_step"]  = float(parts[4])
        elif key == "YDEF":
            meta["nrows"] = int(parts[1])
            meta["y_start"] = float(parts[3])
            meta["y_step"]  = float(parts[4])
        elif key == "UNDEF":
            meta["undef"] = float(parts[1])

ncols, nrows = meta["ncols"], meta["nrows"]
x_start, y_start = meta["x_start"], meta["y_start"]
x_step, y_step = meta["x_step"], meta["y_step"]
nodata_value = meta["undef"]

print(f"Parsed CTL: nrows={nrows}, ncols={ncols}, x_start={x_start}, y_start={y_start}")

# ------------------ 2) Download India district boundaries ------------------
geo_zip_local = os.path.join(GEO_DIR, "geoBoundaries-IND-ADM2-all.zip")
download_file(GEOBOUNDARIES_IND_ADM2_ZIP, geo_zip_local)

with zipfile.ZipFile(geo_zip_local, "r") as z:
    z.extractall(GEO_DIR)
    members = z.namelist()

found_geojson = next((os.path.join(GEO_DIR,m) for m in members if m.lower().endswith(".geojson")), None)
found_shp = next((os.path.join(GEO_DIR,m) for m in members if m.lower().endswith(".shp") and "ADM2" in m), None)

if found_geojson:
    gdf_admin = gpd.read_file(found_geojson)
elif found_shp:
    gdf_admin = gpd.read_file(found_shp)
else:
    cand = [os.path.join(GEO_DIR, f) for f in members if f.lower().endswith((".shp",".geojson"))]
    if not cand: raise RuntimeError("No shapefile/geojson found.")
    gdf_admin = gpd.read_file(cand[0])

if gdf_admin.crs is None or gdf_admin.crs.to_string() != "EPSG:4326":
    gdf_admin = gdf_admin.to_crs(epsg=4326)

possible_name_fields = ["shapeName","shapeName_en","NAME_2","NAME_1","NAME"]
district_name_field = next((f for f in possible_name_fields if f in gdf_admin.columns), None)
if district_name_field is None:
    district_name_field = [c for c in gdf_admin.columns if c != gdf_admin.geometry.name][0]

print("Using district name field:", district_name_field)

# ------------------ 3) Centroid mapping (ensures all districts) ------------------
gdf_admin["centroid"] = gdf_admin.geometry.centroid
gdf_admin["nearest_r"] = ((gdf_admin.centroid.y - y_start) / y_step).round().astype(int)
gdf_admin["nearest_c"] = ((gdf_admin.centroid.x - x_start) / x_step).round().astype(int)

# Clip to valid grid
gdf_admin["nearest_r"] = gdf_admin["nearest_r"].clip(0, nrows-1)
gdf_admin["nearest_c"] = gdf_admin["nearest_c"].clip(0, ncols-1)

# Build mapping dict: district_name -> (r,c)
district_to_grid = dict(zip(gdf_admin[district_name_field], zip(gdf_admin["nearest_r"], gdf_admin["nearest_c"])))
print("Total districts mapped:", len(district_to_grid))

# ------------------ 4) Download all .grd files ------------------
date = start_date
downloaded_files = []
pbar = tqdm(total=(end_date - start_date).days + 1)
while date <= end_date:
    fname = GRD_FILENAME_PATTERN.format(dd=f"{date.day:02d}", mm=f"{date.month:02d}", yyyy=f"{date.year}")
    url = IMD_BASE + fname
    local_path = os.path.join(GRD_DIR, fname)
    r = requests.get(url, stream=True, timeout=30)
    if r.status_code == 200:
        with open(local_path, "wb") as fh:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: fh.write(chunk)
        downloaded_files.append(local_path)
    date += timedelta(days=1)
    pbar.update(1)
pbar.close()
print("Downloaded GRD files:", len(downloaded_files))

if len(downloaded_files) == 0:
    raise RuntimeError("No .grd files downloaded.")

# ------------------ 5) Read .grd files ------------------
def read_grid_best_guess(path):
    try:
        arr = np.fromfile(path, dtype="<f4").reshape((nrows,ncols))
    except:
        arr = np.fromfile(path, dtype=">f4").reshape((nrows,ncols))
    return arr

# ------------------ 6) Aggregate to districts using centroids ------------------
records = []
for local_path in tqdm(sorted(downloaded_files)):
    fname = os.path.basename(local_path)
    dd, mm, yyyy = int(fname[3:5]), int(fname[5:7]), int(fname[7:11])
    dt_str = datetime(yyyy, mm, dd).strftime("%Y-%m-%d")
    arr = read_grid_best_guess(local_path)
    rec = {"date": dt_str}
    for district, (r,c) in district_to_grid.items():
        val = arr[r,c]
        rec[district] = float(np.nan) if val == nodata_value else float(val)
    records.append(rec)

df = pd.DataFrame(records)
cols = ['date'] + [c for c in df.columns if c != 'date']
df = df[cols].sort_values("date").reset_index(drop=True)
df.to_csv(OUT_CSV, index=False)
print("\nSaved combined CSV:", OUT_CSV)
print("CSV shape:", df.shape)

from google.colab import files
files.download(OUT_CSV)


Already have /content/imdpune_min/TEMP.CTL
Parsed CTL: nrows=61, ncols=61, x_start=67.5, y_start=7.5
Already have /content/imdpune_min/geoboundaries/geoBoundaries-IND-ADM2-all.zip
Using district name field: shapeName


/tmp/ipython-input-2352417302.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["centroid"] = gdf_admin.geometry.centroid
/tmp/ipython-input-2352417302.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["nearest_r"] = ((gdf_admin.centroid.y - y_start) / y_step).round().astype(int)
/tmp/ipython-input-2352417302.py:119: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["nearest_c"] = ((gdf_admin.centroid.x - x_start) / x_step).round().astype(int)


Total districts mapped: 728



  5%|▌         | 15/283 [02:46<49:28, 11.08s/it]

100%|██████████| 283/283 [23:23<00:00,  4.96s/it]


Downloaded GRD files: 283


100%|██████████| 283/283 [00:00<00:00, 1069.45it/s]



Saved combined CSV: /content/imdpune_min/district_daily_min_temp_2025.csv
CSV shape: (283, 729)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<font color='green'> <font size ='5'> **Realtime Districts Daily temperature (Maximum) data download for grid data**

In [ ]:
DSET p:\rain_ind0.25_19_03_01.GRD
TITLE 0.25 DEGREE ANALYZED GRIDS
UNDEF  -999.0
XDEF  135  LINEAR 66.5 0.25
YDEF  129  LINEAR 6.5 0.25
ZDEF    1  LINEAR 1 1
TDEF   1  LINEAR JUN2006 1DY
VARS  1
RF 0 99 RFANOMALY
ENDVARS


Downloading: https://www.imdpune.gov.in/cmpg/Realtimedata/max/TEMP.CTL
Saved -> /content/imdpune_max/TEMP.CTL
Parsed CTL: nrows=61, ncols=61, x_start=67.5, y_start=7.5
Downloading: https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/IND/ADM2/geoBoundaries-IND-ADM2-all.zip
Saved -> /content/imdpune_max/geoboundaries/geoBoundaries-IND-ADM2-all.zip


/tmp/ipython-input-829717817.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["centroid"] = gdf_admin.geometry.centroid
/tmp/ipython-input-829717817.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["nearest_r"] = ((gdf_admin.centroid.y - y_start) / y_step).round().astype(int)
/tmp/ipython-input-829717817.py:119: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_admin["nearest_c"] = ((gdf_admin.centroid.x - x_start) / x_step).round().astype(int)


Using district name field: shapeName
Total districts mapped: 728



  0%|          | 0/283 [02:57<?, ?it/s]

100%|██████████| 283/283 [17:49<00:00,  3.78s/it]


Downloaded GRD files: 282


100%|██████████| 282/282 [00:00<00:00, 2034.81it/s]



Saved combined CSV: /content/imdpune_max/district_daily_max_temp_2025.csv
CSV shape: (282, 729)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<font color='blue'> <font size ='5'> **Realtime Districts Daily rainfall data download for grid data**

In [ ]:
# ---------------- Colab-ready: Download IMD Daily Rainfall ----------------
!pip install -q geopandas tqdm shapely fiona bs4 requests

import os, requests, zipfile, glob
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings('ignore')

# ---------------- CONFIG ----------------
IMD_PAGE = "https://imdpune.gov.in/lrfindex.php"
GRD_BASE = "https://imdpune.gov.in/cmpg/Griddata/Rainfall/"
WORKDIR = "/content/imdpune_rain"
GRD_DIR = os.path.join(WORKDIR, "grds")
GEO_DIR = os.path.join(WORKDIR, "geoboundaries")
OUT_CSV = os.path.join(WORKDIR, "district_daily_rain.csv")
os.makedirs(GRD_DIR, exist_ok=True)
os.makedirs(GEO_DIR, exist_ok=True)

# IMD 0.25° grid
ncols, nrows = 135, 129
x_start, y_start = 66.5, 6.5
x_step, y_step = 0.25, 0.25
nodata_value = -999.0

# ---------------- 1) Download district shapefile ----------------
GEOBOUNDARIES_IND_ADM2_ZIP = "https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/IND/ADM2/geoBoundaries-IND-ADM2-all.zip"
geo_zip_local = os.path.join(GEO_DIR, "geoBoundaries-IND-ADM2-all.zip")

if not os.path.exists(geo_zip_local):
    print("Downloading district boundaries...")
    r = requests.get(GEOBOUNDARIES_IND_ADM2_ZIP, stream=True)
    if r.status_code == 200:
        with open(geo_zip_local, "wb") as f:
            for chunk in tqdm(r.iter_content(chunk_size=8192),
                            desc='Downloading boundaries'):
                if chunk:
                    f.write(chunk)

# Extract and find shapefile
shp_file = None
if os.path.exists(geo_zip_local):
    with zipfile.ZipFile(geo_zip_local, "r") as z:
        z.extractall(GEO_DIR)

    for root, dirs, files in os.walk(GEO_DIR):
        for file in files:
            if file.endswith('.shp'):
                shp_file = os.path.join(root, file)
                break
        if shp_file:
            break

if shp_file and os.path.exists(shp_file):
    gdf_admin = gpd.read_file(shp_file)
    gdf_admin_proj = gdf_admin.to_crs(epsg=3857)
    gdf_admin_proj["centroid"] = gdf_admin_proj.geometry.centroid
    gdf_admin_proj = gdf_admin_proj.to_crs(epsg=4326)
    gdf_admin["centroid"] = gdf_admin_proj["centroid"]

    district_name_field = next((f for f in ["shapeName", "shapeName_en", "NAME_2", "district", "District", "NAME_1", "NAME"]
                              if f in gdf_admin.columns), gdf_admin.columns[0])
    print(f"Using district name field: {district_name_field}")
    print(f"Total districts: {len(gdf_admin)}")
else:
    print("Error: Could not find or load shapefile")
    exit()

# ---------------- 2) DOWNLOAD DAILY DATA FROM 1st JAN 2025 TO LATEST ----------------
print("Downloading daily rainfall data from 1st January 2025...")

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_file(year, month, day):
    """Download specific GRD file"""
    yy = str(year)[2:].zfill(2)
    mm = str(month).zfill(2)
    dd = str(day).zfill(2)

    filename = f"rain_ind0.25_{yy}_{mm}_{dd}.grd"
    url = f"{GRD_BASE}{filename}"
    local_path = os.path.join(GRD_DIR, filename)

    # Skip if already exists
    if os.path.exists(local_path):
        return local_path

    try:
        response = requests.get(url, headers=headers, timeout=30)
        if response.status_code == 200:
            # Check if it's a valid binary file
            if len(response.content) > 1000 and b'html' not in response.content[:100].lower():
                with open(local_path, 'wb') as f:
                    f.write(response.content)
                return local_path
    except:
        pass

    return None

# Generate all dates from 1st Jan 2025 to today
start_date = datetime(2025, 1, 1)
end_date = datetime.now()
current_date = start_date

print(f"Checking dates from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")

downloaded_files = []
total_days = (end_date - start_date).days + 1

# Test and download files
for day_offset in tqdm(range(total_days), desc="Checking dates"):
    current_date = start_date + timedelta(days=day_offset)
    file_path = download_file(current_date.year, current_date.month, current_date.day)
    if file_path:
        downloaded_files.append(file_path)

print(f"Downloaded {len(downloaded_files)} daily rainfall files")

# ---------------- 3) Map grid to districts ----------------
print("Mapping districts to grid cells...")
gdf_admin["nearest_r"] = ((gdf_admin.centroid.y - y_start) / y_step).round().astype(int)
gdf_admin["nearest_c"] = ((gdf_admin.centroid.x - x_start) / x_step).round().astype(int)

# Filter valid grid indices
valid_mask = (gdf_admin["nearest_r"] >= 0) & (gdf_admin["nearest_r"] < nrows) & \
             (gdf_admin["nearest_c"] >= 0) & (gdf_admin["nearest_c"] < ncols)

gdf_admin_valid = gdf_admin[valid_mask]
district_to_grid = dict(zip(gdf_admin_valid[district_name_field],
                           zip(gdf_admin_valid["nearest_r"], gdf_admin_valid["nearest_c"])))

print(f"Districts mapped: {len(district_to_grid)}")

# ---------------- 4) Read .GRD file ----------------
def read_grid(path):
    try:
        arr = np.fromfile(path, dtype=np.float32)
        expected_size = nrows * ncols

        if arr.size != expected_size:
            if arr.size > expected_size:
                arr = arr[:expected_size]
            else:
                full_arr = np.full(expected_size, nodata_value, dtype=np.float32)
                full_arr[:arr.size] = arr
                arr = full_arr

        return arr.reshape((nrows, ncols))
    except:
        return np.full((nrows, ncols), nodata_value, dtype=np.float32)

# ---------------- 5) Process files and create CSV ----------------
print("Processing daily rainfall data...")
records = []

# Sort files by date
def extract_date(path):
    filename = os.path.basename(path)
    pattern = r'rain_ind0\.25_(\d+)_(\d+)_(\d+)\.grd'
    match = re.search(pattern, filename)
    if match:
        yy, mm, dd = match.groups()
        return datetime(2000 + int(yy), int(mm), int(dd))
    return datetime(1900, 1, 1)

if downloaded_files:
    downloaded_files.sort(key=extract_date)

    for path in tqdm(downloaded_files, desc="Processing files"):
        try:
            arr = read_grid(path)
            filename = os.path.basename(path)

            pattern = r'rain_ind0\.25_(\d+)_(\d+)_(\d+)\.grd'
            match = re.search(pattern, filename)

            if match:
                yy, mm, dd = match.groups()
                year = 2000 + int(yy)
                date_str = datetime(year, int(mm), int(dd)).strftime("%Y-%m-%d")

                record = {"date": date_str}

                for district, (r, c) in district_to_grid.items():
                    val = arr[r, c]
                    if abs(val - nodata_value) < 0.001 or val < 0:
                        record[district] = float('nan')
                    else:
                        record[district] = float(val)

                records.append(record)

        except:
            continue

# ---------------- 6) Save clean CSV ----------------
if records:
    df = pd.DataFrame(records)

    # Reorder columns: date first, then alphabetical districts
    cols = ["date"] + sorted([c for c in df.columns if c != "date"])
    df = df[cols].sort_values("date").reset_index(drop=True)

    # Save to CSV
    df.to_csv(OUT_CSV, index=False)

    print(f"✅ SUCCESS: Daily rainfall data saved")
    print(f"📅 Dates: {len(records)} days")
    print(f"🏛️  Districts: {len(district_to_grid)}")
    print(f"📁 File: {OUT_CSV}")

    # Show simple sample
    print(f"\nSample data:")
    print(f"First date: {df['date'].min()}")
    print(f"Last date: {df['date'].max()}")
    print(f"Total records: {len(df)}")

else:
    print("❌ No data processed")

# ---------------- 7) Download CSV ----------------
try:
    from google.colab import files
    if os.path.exists(OUT_CSV):
        files.download(OUT_CSV)
        print(f"📥 File downloaded: {OUT_CSV}")
except:
    print(f"💾 File saved: {OUT_CSV}")

print("Done!")

Using district name field: shapeName
Total districts: 735
Checking dates from 2025-01-01 to 2025-10-15...


Checking dates: 100%|██████████| 288/288 [04:26<00:00,  1.08it/s]


Downloaded 4 daily rainfall files
Mapping districts to grid cells...
Districts mapped: 728
Processing daily rainfall data...


Processing files: 100%|██████████| 4/4 [00:00<00:00, 640.13it/s]

✅ SUCCESS: Daily rainfall data saved
📅 Dates: 4 days
🏛️  Districts: 728
📁 File: /content/imdpune_rain/district_daily_rain.csv

Sample data:
First date: 2025-01-15
Last date: 2025-10-01
Total records: 4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File downloaded: /content/imdpune_rain/district_daily_rain.csv
Done!


<font color='green'> <font size ='5'> **Real-time Districts Daily raifall data download for grid data**

In [ ]:
# ================================================
#      IMD RAINFALL SCRIPT — WITH CTL UPLOAD
# ================================================

import os
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point
from google.colab import files

# ------------------------------------------------
# 1) USER UPLOADS CTL FILE
# ------------------------------------------------
print("📌 PLEASE UPLOAD YOUR CTL FILE (e.g., ddmmyyyy.ctl)")
uploaded = files.upload()

CTL_PATH = list(uploaded.keys())[0]
print("CTL file received:", CTL_PATH)

# ------------------------------------------------
# 2) PARSE CTL METADATA
# ------------------------------------------------
meta = {}
with open(CTL_PATH, "r") as f:
    for line in f:
        parts = line.strip().split()
        if not parts:
            continue
        k = parts[0].upper()

        if k == "UNDEF":
            meta["undef"] = float(parts[1])
        elif k == "XDEF":
            meta["ncols"] = int(parts[1])
            meta["x_start"] = float(parts[3])
            meta["x_step"] = float(parts[4])
        elif k == "YDEF":
            meta["nrows"] = int(parts[1])
            meta["y_start"] = float(parts[3])
            meta["y_step"] = float(parts[4])

print("Parsed CTL:", meta)

ncols = meta["ncols"]
nrows = meta["nrows"]
x_start = meta["x_start"]
y_start = meta["y_start"]
x_step = meta["x_step"]
y_step = meta["y_step"]
undef = meta["undef"]

print("\nGRID INFO:")
print("Rows:", nrows, "Cols:", ncols)
print("X start:", x_start, "Step:", x_step)
print("Y start:", y_start, "Step:", y_step)

# ------------------------------------------------
# 3) DOWNLOAD INDIA DISTRICT SHP
# ------------------------------------------------
GEO_URL = "https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/IND/ADM2/geoBoundaries-IND-ADM2-all.zip"
ZIP_PATH = "/content/adm2.zip"

if not os.path.exists(ZIP_PATH):
    print("Downloading district boundaries...")
    r = requests.get(GEO_URL)
    open(ZIP_PATH, "wb").write(r.content)

with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall("/content/adm2/")

geo_file = ""
for f in os.listdir("/content/adm2/"):
    if f.lower().endswith(".geojson"):
        geo_file = "/content/adm2/" + f
        break

gdf = gpd.read_file(geo_file).to_crs(epsg=4326)

# district name field
for col in ["shapeName", "NAME_2", "NAME", "district"]:
    if col in gdf.columns:
        district_field = col
        break

print("District name field:", district_field)

# ------------------------------------------------
# 4) DISTRICT → GRID MAPPING (centroid)
# ------------------------------------------------
gdf_proj = gdf.to_crs(3857)
gdf["centroid"] = gdf_proj.centroid.to_crs(4326)

gdf["row"] = ((gdf.centroid.y - y_start) / y_step).round().astype(int)
gdf["col"] = ((gdf.centroid.x - x_start) / x_step).round().astype(int)

gdf["row"] = gdf["row"].clip(0, nrows - 1)
gdf["col"] = gdf["col"].clip(0, ncols - 1)

district_map = dict(zip(gdf[district_field], zip(gdf["row"], gdf["col"])))

print("Mapped districts:", len(district_map))

# ------------------------------------------------
# 5) AUTO-DOWNLOAD ALL GRD FOR YEAR (2025)
# ------------------------------------------------
RAIN_BASE = "https://www.imdpune.gov.in/cmpg/Realtimedata/Rainfall/"
START = datetime(2025, 1, 1)
END   = datetime(2025, 12, 31)

grd_dir = "/content/grd/"
os.makedirs(grd_dir, exist_ok=True)

def make_filename(d):
    yy = d.year % 100
    return f"rain_ind0.25_{yy:02d}_{d.month:02d}_{d.day:02d}.grd"

downloaded = []
d = START
pbar = tqdm(total=(END - START).days + 1, desc="Downloading GRD")

while d <= END:
    fname = make_filename(d)
    url = RAIN_BASE + fname
    local = grd_dir + fname

    r = requests.get(url)
    if r.status_code == 200:
        open(local, "wb").write(r.content)
        downloaded.append(local)
    else:
        print("Missing:", fname)

    d += timedelta(days=1)
    pbar.update(1)

pbar.close()
print("Total GRD downloaded:", len(downloaded))

# ------------------------------------------------
# 6) READ GRD FILE
# ------------------------------------------------
def read_grd(path):
    try:
        return np.fromfile(path, dtype="<f4").reshape((nrows, ncols))
    except:
        return np.fromfile(path, dtype=">f4").reshape((nrows, ncols))

# ------------------------------------------------
# 7) PROCESS ALL DOWNLOADED GRD
# ------------------------------------------------
rows = []

for path in tqdm(downloaded, desc="Processing GRDs"):
    fname = os.path.basename(path).replace(".grd", "")
    parts = fname.split("_")
    # ['rain', 'ind0.25', '25', '07', '01']

    yy = int(parts[2])
    mm = int(parts[3])
    dd = int(parts[4])
    yyyy = 2000 + yy
    date_str = f"{yyyy:04d}-{mm:02d}-{dd:02d}"

    arr = read_grd(path)
    rec = {"date": date_str}

    for dist, (r, c) in district_map.items():
        v = arr[r, c]
        rec[dist] = np.nan if v == undef else float(v)

    rows.append(rec)

df = pd.DataFrame(rows).sort_values("date")

# ------------------------------------------------
# 8) SAVE CSV
# ------------------------------------------------
OUT = "/content/IMD_Rainfall_Districtwise_2025.csv"
df.to_csv(OUT, index=False)

print("\n🎉 DONE! Your final CSV is ready:")
print(OUT)
files.download(OUT)
